In [ ]:
## 分析同源基因bias表达
rm(list=ls())
#setwd('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/reads')
setwd('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/homo_gene2/gene_bias/DESeq')
library(DESeq2)
library(dplyr)
#tissues <- list('anthers', 'cotyledon',  'hypocotyl', 'leaf', 'ovules', 'petals', 'radicle', 'root', 'stem', 'stigma')
tissues <- list('fiber_5DPA', 'fiber_10DPA', 'fiber_20DPA')
#tissues <- list('anthers')
for (t in tissues){
    print(t)
    sample = paste(t, "/", t, '_hg_reads.bed', sep="")
    out_f = paste(t, "/", t, '_DEG.bed', sep="")
    countData <- read.table(sample, sep='\t', header=T, row.names=1)
    #countData <- select(countData, reads1_at, reads2_at, reads3_at, reads1_dt, reads2_dt, reads3_dt)
    countData <- select(countData, reads1_at, reads2_at, reads1_dt, reads2_dt)
    # print(countData)
    #condition <- factor(c("at", "at", "at", "dt", "dt", "dt"))
    condition <- factor(c("at", "at", "dt", "dt"))
    colData <- data.frame(row.names=colnames(countData), condition)
    all(rownames(colData)) == colnames(countData)
    dds <- DESeqDataSetFromMatrix(countData, DataFrame(colData), design=~condition)
    #print(dds)
    # #View(dds)
    dds <- DESeq(dds)
    resultsNames(dds)
    res <- results(dds)
    # table(res$padj==0)
    # mcols(res, use.names=TRUE)
    res <- res[order(res$padj), ]
    res <- subset(res, padj != 0)
    GeneId <- row.names(res)
    logFC <- res$log2FoldChange
    padj <- res$padj
    data <- data.frame(GeneId=GeneId, logFC=logFC, padj=padj)
    data$sig[(data$padj>0.01)|((data$logFC < 1)&(data$logFC> -1))] <- "Not"
    data$sig[data$padj <= 0.01 & data$logFC >= 1] <- "Up"
    data$sig[data$padj <= 0.01 & data$logFC <= -1] <- "Down"
    data_out = subset(data, sig != 'Not')
    write.table(data_out, file=out_f, sep='\t', row.names=F)
} 

In [ ]:
## 分析同源基因bias表达
rm(list=ls())
setwd('G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/RNA_Seq_data')
library(DESeq2)
library(dplyr)
tissues <- list('anther', 'cotyledon',  'hypocotyl', 'leaf', 'ovule', 'petal', 'radicle', 'root', 'stem', 'stigma', 'fiber_10DPA', 'fiber_20DPA')
batch1_sample <- c('anther', 'hypocotyl')
batch2_sample <- c('cotyledon', 'leaf', 'ovule', 'petal', 'root', 'stem', 'stigma')
batch3_sample <- c('radicle', 'fiber_10DPA', 'fiber_20DPA')
df <- read.csv("J668_all_Tissue_hg_reads.csv",header=TRUE, row.names=1)
#print(df)
for (t in tissues){
    at_rep1 = paste(t, "_rep1_at", sep="")
    at_rep2 = paste(t, "_rep2_at", sep="")
    at_rep3 = paste(t, "_rep3_at", sep="")
    dt_rep1 = paste(t, "_rep1_dt", sep="")
    dt_rep2 = paste(t, "_rep2_dt", sep="")
    dt_rep3 = paste(t, "_rep3_dt", sep="")
    out_f = paste(t, '_DEG.bed', sep="")
    #print(at_rep1)
    countData <- select(df, at_rep1, at_rep2, at_rep3, dt_rep1, dt_rep2, dt_rep3)
    #rename
    names(countData)[names(countData) == at_rep1] <- "Control1"
    names(countData)[names(countData) == at_rep2] <- "Control2"
    names(countData)[names(countData) == at_rep3] <- "Control3"
    names(countData)[names(countData) == dt_rep1] <- "Treatment1"
    names(countData)[names(countData) == dt_rep2] <- "Treatment2"
    names(countData)[names(countData) == dt_rep3] <- "Treatment3"
    # 批次效应
    if (t %in% batch1_sample) {
        at_batch_sample <- c(1, 2, 2)
        dt_batch_sample <- c(1, 2, 2)
    } else if (t %in% batch2_sample) {
        at_batch_sample <- c(1, 1, 2)
        dt_batch_sample <- c(1, 1, 2)
    } else {
        at_batch_sample <- c(1, 1, 3)
        dt_batch_sample <- c(1, 1, 3)
    }
    batch <- factor(c(at_batch_sample, dt_batch_sample))
    # # # print(countData)
    condition <- factor(c("Control", "Control", "Control", "Treatment", "Treatment", "Treatment"))
    #colData <- data.frame(row.names=colnames(countData), condition)
    colData <- data.frame(condition=condition, batch=batch)
    all(rownames(colData)) == colnames(countData)
    dds <- DESeqDataSetFromMatrix(countData, DataFrame(colData), design=~batch+condition)
    #print(dds)
    # #View(dds)
    dds <- DESeq(dds)
    resultsNames(dds)
    res <- results(dds)
    # table(res$padj==0)
    # mcols(res, use.names=TRUE)
    res <- res[order(res$padj), ]
    res <- subset(res, padj != 0)
    GeneId <- row.names(res)
    logFC <- res$log2FoldChange
    padj <- res$padj
    data <- data.frame(GeneId=GeneId, logFC=logFC, padj=padj)
    data$sig[(data$padj>0.01)|((data$logFC < 1)&(data$logFC> -1))] <- "Not"
    data$sig[data$padj <= 0.01 & data$logFC >= 1] <- "Up"
    data$sig[data$padj <= 0.01 & data$logFC <= -1] <- "Down"
    data_out = subset(data, sig != 'Not')
    write.table(data_out, file=out_f, sep='\t', row.names=F)
} 

In [ ]:
## 分析fiber_5DPA 和fiber_10DPA之间差异表达的基因，
rm(list=ls())
#setwd('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/reads')
setwd('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/reads/5DPA_vs_10DPA')
library(DESeq2)
library(dplyr)
countData <- read.table("5DPA_10DPA_reads.bed", sep='\t', header=T, row.names=1)
countData <- select(countData, control1, control2, KD1, KD2)
condition <- factor(c("control", "control", "KD", "KD"))
colData <- data.frame(row.names=colnames(countData), condition)
all(rownames(colData)) == colnames(countData)
dds <- DESeqDataSetFromMatrix(countData, DataFrame(colData), design=~condition)
dds <- DESeq(dds)
resultsNames(dds)
res <- results(dds)
res <- res[order(res$padj), ]
res <- subset(res, padj != 0)
GeneId <- row.names(res)
logFC <- res$log2FoldChange
padj <- res$padj
data <- data.frame(GeneId=GeneId, logFC=logFC, padj=padj)
data$sig[(data$padj>0.01)|((data$logFC < 1)&(data$logFC> -1))] <- "Not"
data$sig[data$padj <= 0.01 & data$logFC >= 1] <- "Up"
data$sig[data$padj <= 0.01 & data$logFC <= -1] <- "Down"
data_out = subset(data, sig != 'Not')
write.table(data_out, file="5DPA_10DPA_DEG.bed", sep='\t', row.names=F)


In [ ]:
## 分析ovules 和fiber_5DPA之间差异表达的基因，
rm(list=ls())
setwd('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/reads/10DPA_vs_20DPA')
library(DESeq2)
library(dplyr)
countData <- read.table("10DPA_20DPA_reads.bed", sep='\t', header=T, row.names=1)
countData <- select(countData, CT1, CT2, KD1, KD2)
condition <- factor(c("CT", "CT", "KD", "KD"))
colData <- data.frame(row.names=colnames(countData), condition)
all(rownames(colData)) == colnames(countData)
dds <- DESeqDataSetFromMatrix(countData, DataFrame(colData), design=~condition)
dds <- DESeq(dds)
resultsNames(dds)
res <- results(dds)
res <- res[order(res$padj), ]
res <- subset(res, padj != 0)
GeneId <- row.names(res)
logFC <- res$log2FoldChange
padj <- res$padj
data <- data.frame(GeneId=GeneId, logFC=logFC, padj=padj)
data$sig[(data$padj>0.01)|((data$logFC < 1)&(data$logFC> -1))] <- "Not"
data$sig[data$padj <= 0.01 & data$logFC >= 1] <- "Up"
data$sig[data$padj <= 0.01 & data$logFC <= -1] <- "Down"
data_out = subset(data, sig != 'Not')
write.table(data_out, file="10DPA_20DPA_DEG.bed", sep='\t', row.names=F)


In [ ]:
## 分析fiber_5DPA 和fiber_20DPA之间差异表达的基因，
rm(list=ls())
#setwd('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/reads')
setwd('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/reads/5DPA_vs_20DPA')
library(DESeq2)
library(dplyr)
countData <- read.table("5DPA_20DPA_reads.bed", sep='\t', header=T, row.names=1)
countData <- select(countData, DPA5_rep1, DPA5_rep2, DPA20_rep1, DPA20_rep2)
condition <- factor(c("DPA5", "DPA5", "DPA20", "DPA20"))
colData <- data.frame(row.names=colnames(countData), condition)
all(rownames(colData)) == colnames(countData)
dds <- DESeqDataSetFromMatrix(countData, DataFrame(colData), design=~condition)
dds <- DESeq(dds)
resultsNames(dds)
res <- results(dds)
res <- res[order(res$padj), ]
res <- subset(res, padj != 0)
GeneId <- row.names(res)
logFC <- res$log2FoldChange
padj <- res$padj
data <- data.frame(GeneId=GeneId, logFC=logFC, padj=padj)
data$sig[(data$padj>0.01)|((data$logFC < 1)&(data$logFC> -1))] <- "Not"
data$sig[data$padj <= 0.01 & data$logFC >= 1] <- "Up"
data$sig[data$padj <= 0.01 & data$logFC <= -1] <- "Down"
data_out = subset(data, sig != 'Not')
write.table(data_out, file="5DPA_20DPA_DEG.bed", sep='\t', row.names=F)


In [ ]:
## 比较组织间两两差异
library(DESeq2)
library(dplyr)
setwd("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/RNA_Seq_data")
tissues = c('anther', 'ovule', 'root',  'radicle', 'leaf', 'hypocotyl', 'cotyledon', 'stem', 'fiber_10DPA', 'fiber_20DPA', 'petal', 'stigma')
df = read.csv("all_tissues_reads.bed",  header=T, row.names=1)
#print(df)
for (t1 in seq(1, 9)) {
    one <- tissues[t1]
    if (one == "Anther" | one== "Hypocotyl") {
        ac <- c(1,2,2)
    } else {
        ac <- c(1,1,2)
    }
    for (t2 in seq(t1+1,10)) {
        two <- tissues[t2]
            if (two == "Anther" | two == "Hypocotyl") {
            bc <- c(1,2,2)
        } else {
            bc <- c(1,1,2)
        }
        out_f <- paste(one, "_", two, "_DEG.bed", sep="")
        c1 = paste(one, 1, sep="")
        c2 = paste(one, 2, sep="")
        c3 = paste(one, 3, sep="")
        t1 = paste(two, 1, sep="")
        t2 = paste(two, 2, sep="")
        t3 = paste(two, 3, sep="")
        control_treatment = c(c1,c2,c3,t1,t2,t3)
        print(control_treatment)
        #print(control_treatment)
        countData = select(df, control_treatment)
        condition <- factor(c("control", "control", "control", "treat", "treat", "treat"))
        batch <- factor(c(ac, bc))
        #colData <- data.frame(row.names=colnames(countData), condition)
        colData <- data.frame(condition=condition,batch=batch)
        all(rownames(colData)) == colnames(countData)
        dds <- DESeqDataSetFromMatrix(countData, DataFrame(colData), design=~batch+condition)
        #View(dds)
        dds <- DESeq(dds)
        resultsNames(dds)
        res <- results(dds)
        table(res$padj==0)
        mcols(res, use.names=TRUE)
        res <- res[order(res$padj), ]
        res <- subset(res, padj != 0)
        GeneId <- row.names(res)
        logFC <- res$log2FoldChange
        padj <- res$padj
        data <- data.frame(GeneId=GeneId, logFC=logFC, padj=padj)
        data$sig[(data$padj>0.05)|((data$logFC < 1)&(data$logFC> -1))] <- "Not"
        data$sig[data$padj <= 0.05 & data$logFC >= 1] <- "Up"
        data$sig[data$padj <= 0.05 & data$logFC <= -1] <- "Down"
        #data_out = subset(data, sig != 'Not')
        write.table(data, file=out_f, sep='\t', row.names=F)
    }
}

In [ ]:
## 比较两两组织间两两差异
library(DESeq2)
library(dplyr)
setwd("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/RNA_Seq_data")
tissues = c('anther', 'ovule', 'root',  'radicle', 'leaf', 'hypocotyl', 'cotyledon', 'stem', 'fiber_10DPA', 'fiber_20DPA', 'petal', 'stigma')
df = read.csv("J668_all_Tissue_reads.csv",  header=T, row.names=1)
batch1_sample <- c('anther', 'hypocotyl')
batch2_sample <- c('cotyledon', 'leaf', 'ovule', 'petal', 'root', 'stem', 'stigma')
batch3_sample <- c('radicle', 'fiber_10DPA', 'fiber_20DPA')
#print(df)
for (t1 in seq(1, 11)) {
    one <- tissues[t1]
    if (t1 %in% batch1_sample) {
        control_batch_sample <- c(1, 2, 2)
    } else if (t1 %in% batch2_sample) {
        control_batch_sample <- c(1, 1, 2)
    } else {
        control_batch_sample <- c(1, 1, 3)
    }

    for (t2 in seq(t1+1,12)) {
        if (t2 %in% batch1_sample) {
            treatment_batch_sample <- c(1, 2, 2)
        } else if (t2 %in% batch2_sample) {
            treatment_batch_sample <- c(1, 1, 2)
        } else {
            treatment_batch_sample <- c(1, 1, 3)
        }
        two <- tissues[t2]
        out_f <- paste(one, "_", two, "_DEG.csv", sep="")
        print(out_f)
        c1 = paste(one, "_rep1", sep="")
        c2 = paste(one, "_rep2", sep="")
        c3 = paste(one, "_rep3", sep="")
        t1 = paste(two, "_rep1", sep="")
        t2 = paste(two, "_rep2", sep="")
        t3 = paste(two, "_rep3", sep="")
        control_treatment = c(c1,c2,c3,t1,t2,t3)
        print(control_treatment)
        #print(control_treatment)
        countData = select(df, control_treatment)
        print(head(countData))
            #rename
        names(countData)[names(countData) == c1] <- "Control1"
        names(countData)[names(countData) == c2] <- "Control2"
        names(countData)[names(countData) == c3] <- "Control3"
        names(countData)[names(countData) == t1] <- "Treatment1"
        names(countData)[names(countData) == t2] <- "Treatment2"
        names(countData)[names(countData) == t3] <- "Treatment3"        
        condition <- factor(c("Control", "Control", "Control", "Treat", "Treat", "Treat"))
        batch <- factor(c(control_batch_sample, treatment_batch_sample))
        #colData <- data.frame(row.names=colnames(countData), condition)
        colData <- data.frame(condition=condition,batch=batch)
        all(rownames(colData)) == colnames(countData)
        dds <- DESeqDataSetFromMatrix(countData, DataFrame(colData), design=~batch+condition)
        #View(dds)
        dds <- DESeq(dds)
        resultsNames(dds)
        res <- results(dds)
        table(res$padj==0)
        mcols(res, use.names=TRUE)
        res <- res[order(res$padj), ]
        res <- subset(res, padj != 0)
        GeneId <- row.names(res)
        logFC <- res$log2FoldChange
        padj <- res$padj
        data <- data.frame(GeneId=GeneId, logFC=logFC, padj=padj)
        data$sig[(data$padj>0.01)|((data$logFC < 1)&(data$logFC> -1))] <- "Not"
        data$sig[data$padj <= 0.01 & data$logFC >= 1] <- "Up"
        data$sig[data$padj <= 0.01 & data$logFC <= -1] <- "Down"
        #data_out = subset(data, sig != 'Not')
        write.csv(data, file=out_f,row.names=F)
    }
}

In [15]:
## 分析同源差异表达基因
library(DESeq2)
library(dplyr)
#setwd("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/DEG/tissue_hg/AA")
#setwd("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/DEG/tissue_hg/AD")
#setwd("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/DEG/tissue_hg/DD")
setwd("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/DEG/tissue_hg/DA")

tissues = c('anther', 'ovule', 'root',  'radicle', 'leaf', 'hypocotyl', 'cotyledon', 'stem', 'fiber_10DPA', 'fiber_20DPA', 'petal', 'stigma')
batch1_sample <- c('anther', 'hypocotyl')
batch2_sample <- c('cotyledon', 'leaf', 'ovule', 'petal', 'root', 'stem', 'stigma')
batch3_sample <- c('radicle', 'fiber_10DPA', 'fiber_20DPA')
for (t1 in seq(1, 11)) {
    one <- tissues[t1]
    if (t1 %in% batch1_sample) {
        control_batch_sample <- c(1, 2, 2)
    } else if (t1 %in% batch2_sample) {
        control_batch_sample <- c(1, 1, 2)
    } else {
        control_batch_sample <- c(1, 1, 3)
    }
    for (t2 in seq(t1+1,12)) {
        two <- tissues[t2]
        if (t2 %in% batch1_sample) {
            treatment_batch_sample <- c(1, 2, 2)
        } else if (t2 %in% batch2_sample) {
            treatment_batch_sample <- c(1, 1, 2)
        } else {
            treatment_batch_sample <- c(1, 1, 3)
        }
        input_f <- paste(one, "_", two, ".bed", sep="")
        out_f <- paste(one, "_", two, "_DEG.bed", sep="")
        df = read.table(input_f, sep="\t", header=T, row.names=1)
        countData <- as.matrix(df)
        condition <- factor(c("control", "control", "control", "treat", "treat", "treat"))
        batch <- factor(c(control_batch_sample, treatment_batch_sample))
        #colData <- data.frame(row.names=colnames(countData), condition)
        colData <- data.frame(condition=condition,batch=batch)
        all(rownames(colData)) == colnames(countData)
        # colData <- data.frame(row.names=colnames(countData), condition)
        # all(rownames(colData)) == colnames(countData)
        dds <- DESeqDataSetFromMatrix(countData, DataFrame(colData), design=~batch+condition)
        #View(dds)
        dds <- DESeq(dds)
        resultsNames(dds)
        res <- results(dds)
        table(res$padj==0)
        mcols(res, use.names=TRUE)
        res <- res[order(res$padj),]
        res <- subset(res, padj != 0)
        GeneId <- row.names(res)
        logFC <- res$log2FoldChange
        padj <- res$padj
        data <- data.frame(GeneId=GeneId, logFC=logFC, padj=padj)
        data$sig[(data$padj>0.01)|((data$logFC < 1)&(data$logFC> -1))] <- "Not"
        data$sig[data$padj <= 0.01 & data$logFC >= 1] <- "Up"
        data$sig[data$padj <= 0.01 & data$logFC <= -1] <- "Down"
        #data_out = subset(data, sig != 'Not')
        write.csv(data, file=out_f, row.names=F)
    }
}



Warning message in all(rownames(colData)):
"coercing argument of type 'character' to logical"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Warning message in all(rownames(colData)):
"coercing argument of type 'character' to logical"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Warning message in all(rownames(colData)):
"coercing argument of type 'character' to logical"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Warning message in all(rownames(colData)):
"coercing argument of type 'character' to logical"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersio

In [3]:
browseVignettes("HTSFilter")

starting httpd help server ...
 done



[1] "d"
